In [ ]:
import tensorflow as tf
tf.__version__
#!pip uninstall tensorflow
#!pip install tensorflow
#%tensorflow_version 1.x

import keras
print(keras.__version__)
!pip install keras==2.14.0


In [ ]:
# 3D TRAIN AND VAL Data generator

from google.colab import drive
import os
import random
import numpy as np
from PIL import Image

from PIL import Image
import numpy as np

def load_image(file_path):
    image = Image.open(file_path)

    # Crop in square form
    image = image.crop((0, 0, 256, 256))

    # Convert image to NumPy array
    image_array = np.array(image)

    return image_array

def train_val_data_generator(blur_folder, crisp_folder, mini_stack_height, batch_size):

    validation_split = 0.05
    num_validation = int(validation_split * batch_size) + 1
    num_train = batch_size - num_validation

    # Infinite loop to yield batches during training
    while True: # Random selection of images in multiple folders

        # Initialize lists to store mini-stacks and corresponding crisp images
        X_batch = []
        Y_batch = []
        count = 0;
        print(f"Generating 2D pairs...")
        # Iterate over the indices in the batch
        while count != batch_size:
            count+=1

            folders = ["480490_541480", "500480_533880", "500484_508050", "533470_523780"]

            # Define the mapping of folders to num_images
            folder_to_num_images = {
                "480490_541480": 2408,
                "500480_533880": 2888,
                "500484_508050": 2888,
                "533470_523780": 2862
            }
            random_folder = random.choice(folders)

            num_images = folder_to_num_images.get(random_folder, 0)

            random_ind = random.randint(200, num_images-200)

            crisp_image = load_image(os.path.join(crisp_folder, f"{random_folder}", f"{random_folder}_{random_ind:04d}.tif"))
            blur_mini_stack = [
                            load_image(os.path.join(blur_folder, f"{random_folder}", f"{random_ind - mini_stack_height // 2 + j:05d}.tif"))
                            for j in range((mini_stack_height))
                        ]


            X_batch.append(np.stack(blur_mini_stack, axis=0))
            Y_batch.append(crisp_image)

            #print(f": {count}/{batch_size}, {random_folder}_{random_ind:04d}.tif")

        X_train, Y_train = X_batch[:num_train], Y_batch[:num_train]
        X_val, Y_val = X_batch[num_train:], Y_batch[num_train:]


        print(f"Shape of X_train: {X_train[0].shape}")
        print(f"Shape of Y_train: {Y_train[0].shape}")

        print(f"Image pairs prepared. Train/Val = {len(X_train)}/{len(X_val)}")

        yield (X_train, Y_train), (X_val, Y_val)


drive.mount('/content/gdrive')
train_blur_folder = "gdrive/My Drive/Noisy_data/"
train_crisp_folder = "gdrive/My Drive/Crisp_data/"


batch_size = 3

mini_stack_height = 4

train_val_data = train_val_data_generator(train_blur_folder, train_crisp_folder, mini_stack_height, batch_size)

(X_train, Y_train), (X_val, Y_val) = next(train_val_data)

X_min_value = np.min(X_train[0])
X_max_value = np.max(X_train[0])
Y_min_value = np.min(Y_train[0])
Y_max_value = np.max(Y_train[0])

print(f"X_train min/max Value: {X_min_value}/{X_max_value}")
print(f"Y_train min/max Value: {Y_min_value}/{Y_max_value}")




In [ ]:
# 3D UNet USING TRAIN VAL GENERATOR

import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from google.colab import drive
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam

import os
import pandas as pd
import numpy as np

import PIL
import PIL.Image
import pathlib
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import io
import random

batch_size = 3
mini_stack_height = 16


################################################################################
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import Reshape


def unet_3d():

    inputs = layers.Input(shape=(16, 256, 256, 1))
    conv1 = layers.Conv3D(filters=32, activation="relu", padding="same", kernel_size=3)(inputs)
    conv2 = layers.Conv3D(filters=32, activation="relu", padding="same", kernel_size=3)(conv1)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = layers.Conv3D(filters=64, activation="relu", padding="same", kernel_size=3)(pool1)
    conv4 = layers.Conv3D(filters=64, activation="relu", padding="same", kernel_size=3)(conv3)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)

    conv5 = layers.Conv3D(filters=128, activation="relu", padding="same", kernel_size=3)(pool2)
    conv6 = layers.Conv3D(filters=128, activation="relu", padding="same", kernel_size=3)(conv5)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv6)

    conv7 = layers.Conv3D(filters=256, activation="relu", padding="same", kernel_size=3)(pool3)
    conv8 = layers.Conv3D(filters=256, activation="relu", padding="same", kernel_size=3)(conv7)

    up1 = layers.UpSampling3D(size=(2, 2, 2))(conv8)
    conc1 = layers.Concatenate()([conv6, up1])

    conv9 = layers.Conv3D(filters=128, activation="relu", padding="same", kernel_size=3)(conc1)
    conv10 = layers.Conv3D(filters=128, activation="relu", padding="same", kernel_size=3)(conv9)

    up2 = layers.UpSampling3D(size=(2, 2, 2))(conv10)
    conc2 = layers.Concatenate()([conv4, up2])

    conv11 = layers.Conv3D(filters=64, activation="relu", padding="same", kernel_size=3)(conc2)
    conv12 = layers.Conv3D(filters=64, activation="relu", padding="same", kernel_size=3)(conv11)

    up3 = layers.UpSampling3D(size=(2, 2, 2))(conv12)
    conc3 = layers.Concatenate()([conv2, up3])

    conv13 = layers.Conv3D(filters=32, activation="relu", padding="same", kernel_size=3)(conc3)
    conv14 = layers.Conv3D(filters=32, activation="relu", padding="same", kernel_size=3)(conv13)

    # Add an additional pooling layer to reduce the height dimension to 1
    pool4 = layers.MaxPooling3D(pool_size=(16, 1, 1))(conv14)

    # Change the last Conv3D layer to Conv2D
    output = layers.Conv2D(filters=1, kernel_size=1, activation=None)(pool4)

    # Reshape the output to be 2D
    output = layers.Reshape((256, 256, 1))(output)

    model = keras.Model(inputs=[inputs], outputs=[output])
    return model


model = unet_3d()

loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
#model.compile(optimizer=optimizer, loss=loss_fn, metrics=["mse"])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

################################################################################


# Train the model using the data generator
train_val_data = train_val_data_generator(train_blur_folder, train_crisp_folder, mini_stack_height, batch_size)

psnr_history = []
ssim_history = []
pcc_history = []

# Training loop
epochs = 2000  # Adjust as needed
for epoch in range(epochs):
    (X_train, Y_train), (X_val, Y_val) = next(train_val_data)

    print(f'Epoch {epoch + 1}/{epochs}')

    # Train the model
    history = model.fit(np.array(X_train), np.array(Y_train), epochs=1, validation_data=(np.array(X_val), np.array(Y_val)))

    # Evaluate the model on the validation set
    val_preds = model.predict(np.array(X_val))

    val_preds_min_value = np.min(val_preds[0])
    val_preds_max_value = np.max(val_preds[0])
    gt_min_value = np.min(Y_val[0])
    gt_max_value = np.max(Y_val[0])

    print(f"val_preds Min/Max Value: {val_preds_min_value} / {val_preds_max_value}")
    print(f"ground-truth Min/Max Value: {gt_min_value} / {gt_max_value}")

    # Calculate and print PSNR and SSIM for the test set
    psnr_val_list = [psnr(np.squeeze(Y_val[i]), np.squeeze(val_preds[i]), data_range=65535.0) for i in range(len(Y_val))]
    ssim_val_list = [ssim(np.squeeze(Y_val[i]), np.squeeze(val_preds[i]), data_range=65535.0) for i in range(len(Y_val))]
    pcc_val_list = [pearsonr(np.squeeze(Y_val[i]).ravel(), np.squeeze(val_preds[i]).ravel())[0] for i in range(len(Y_val))]

    # Report the average PSNR, SSIM, and PCC for the test set
    average_psnr_val = np.mean(psnr_val_list)
    average_ssim_val = np.mean(ssim_val_list)
    average_pcc_val = np.mean(pcc_val_list)

    # Store PSNR, SSIM, and PCC values in history lists
    psnr_history.append(average_psnr_val)
    ssim_history.append(average_ssim_val)
    pcc_history.append(average_pcc_val)

    # Plotting real-time PSNR and SSIM
    plt.figure(figsize=(12, 4))

    # Plot PSNR
    plt.subplot(1, 3, 1)
    plt.plot(psnr_history, label='PSNR')
    plt.xlabel('Epoch')
    plt.ylabel('PSNR')
    plt.title('Real-time PSNR during Validation')
    plt.legend()

    # Plot SSIM
    plt.subplot(1, 3, 2)
    plt.plot(ssim_history, label='SSIM', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('SSIM')
    plt.title('Real-time SSIM during Validation')
    plt.legend()

     # Plot PCC
    plt.subplot(1, 3, 3)
    plt.plot(pcc_history, label='PCC', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('PCC')
    plt.title('Real-time PCC during Validation')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print(f'Epoch {epoch + 1}/{epochs} - PSNR: {average_psnr_val}, SSIM: {average_ssim_val}')

    # Display the original (noisy) image
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(X_val[0][mini_stack_height//2]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Original (Noisy) Image')
    plt.axis('off')

    # Display the predicted (crisp) image
    plt.subplot(1, 3, 2)
    plt.imshow(np.squeeze(val_preds[0]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Predicted (Crisp) Image')
    plt.axis('off')

    # Display the ground truth image
    plt.subplot(1, 3, 3)
    plt.imshow(np.squeeze(Y_val[0]) * 1.0, cmap='gray', vmin=0, vmax=500)
    plt.title('Ground Truth Image')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


from datetime import datetime

# Get the current date and time
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
# Define the model file name with the current time
model_file_name = f'model_ds_{current_time}.keras'
model_save_path = f'gdrive/My Drive/{model_file_name}'

# Save the model to Google Drive
save_model(model, model_save_path)
print(f'Model saved to {model_save_path}')


In [ ]:
# 3D TEST Data generator (RANDOM NOISE)

from google.colab import drive
import os
import random
import numpy as np
from PIL import Image

def load_image(file_path):
    image = Image.open(file_path)

    # Crop in square form
    image = image.crop((244, 144, 500, 400))

    # Convert image to NumPy array
    image_array = np.array(image)

    return image_array

def test_data_generator(blur_folder, crisp_folder, mini_stack_height, batch_size):

    # Infinite loop to yield batches during training
    while True: # Random selection of images in multiple folders

        # Initialize lists to store mini-stacks and corresponding crisp images
        X_test = []
        Y_test = []
        count = 0;
        print(f"Generating TEST 3D stacks...")
        # Iterate over the indices in the batch
        while count != batch_size:
            count+=1

            random_ind = random.randint(800, 2000)

            print(f": {count}/{batch_size}, {random_ind:05d}.tif")

            crisp_image = load_image(os.path.join(crisp_folder, f"500480_482220_{random_ind:04d}.tif"))


            noisy_mini_stack = [
                np.clip(
                    load_image(os.path.join(crisp_folder, f"500480_482220_{random_ind - mini_stack_height // 2 + j:04d}.tif"))
                    + np.random.normal(loc=0, scale=random.randint(30, 40), size=crisp_image.shape),
                    0,
                    65535.0
                )
                for j in range(mini_stack_height)
            ]

            # Assuming 16-bit images, changing the scale to [0,1]
            X_test.append(np.stack(noisy_mini_stack, axis=0))
            Y_test.append(crisp_image)


        print(f"Shape of X_train: {X_test[0].shape}")
        print(f"Shape of Y_train: {Y_test[0].shape}")
        print(f"TEST pairs prepared. Test length = {len(X_test)}")

        yield X_test, Y_test


drive.mount('/content/gdrive')
test_blur_folder = "gdrive/My Drive/Noisy_data/500480_482220/"
test_crisp_folder = "gdrive/My Drive/Crisp_data/500480_482220/"


batch_size = 3
mini_stack_height = 4

test_data = test_data_generator(test_blur_folder, test_crisp_folder, mini_stack_height, batch_size)

X_test, Y_test = next(test_data)

X_min_value = np.min(X_test[0])
X_max_value = np.max(X_test[0])
Y_min_value = np.min(Y_test[0])
Y_max_value = np.max(Y_test[0])

print(f"X_test min/max Value: {X_min_value}/{X_max_value}")
print(f"Y_test min/max Value: {Y_min_value}/{Y_max_value}")


In [ ]:
# TEST, compare 2D and 3D CRISPIM
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import pearsonr
import cv2
import matplotlib.pyplot as plt
from skimage.restoration import denoise_tv_bregman


# Load the saved model from Google Drive
model_2d_path = f'gdrive/My Drive/model_ds_20231203_182307.keras'
model_3d_path = f'gdrive/My Drive/model_ds_20231213_180605.keras'
crispim_2d = load_model(model_2d_path)
crispim_3d = load_model(model_3d_path)

batch_size = 20
mini_stack_height = 16

test_data = test_data_generator(test_blur_folder, test_crisp_folder, mini_stack_height, batch_size)
X_test, Y_test = next(test_data)
X_test_single_im = [stack[mini_stack_height//2] for stack in X_test]

# Use the loaded model to predict the crisp image
test_pred_2d = crispim_2d.predict(np.array(X_test_single_im))
test_pred_3d = crispim_3d.predict(np.array(X_test))

#################################################################################

# Define functions for metrics calculation
def calculate_metrics(original, denoised):
    psnr_value = psnr(np.squeeze(original), np.squeeze(denoised), data_range=65535.0)
    ssim_value = ssim(np.squeeze(original), np.squeeze(denoised), data_range=65535.0)
    pcc_value, _ = pearsonr(original.flatten(), denoised.flatten())
    return psnr_value, ssim_value, pcc_value

# Function to display images and calculate metrics
def display_and_calculate_metrics(original, predicted_2d, predicted_3d, ground_truth):
    # Display images
    plt.figure(figsize=(30, 10))

    plt.subplot(1, 4, 1)
    plt.imshow(original, cmap='gray', vmin=0, vmax=500)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(predicted_2d, cmap='gray', vmin=0, vmax=500)
    plt.title('2D CRISPIM')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(predicted_3d, cmap='gray', vmin=0, vmax=500)
    plt.title('3D CRISPIM')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(ground_truth, cmap='gray', vmin=0, vmax=500)
    plt.title('Ground Truth')
    plt.axis('off')

    # Calculate and print metrics
    psnr_2d, ssim_2d, pcc_2d = calculate_metrics(ground_truth, predicted_2d)
    psnr_3d, ssim_3d, pcc_3d = calculate_metrics(ground_truth, predicted_3d)


# Lists to store metric values for each image
psnr_2d_list, ssim_2d_list, pcc_2d_list = [], [], []
psnr_3d_list, ssim_3d_list, pcc_3d_list = [], [], []

# Iterate through batches
num_batches = len(X_test)

# Calculate metrics for each batch
for batch_index in range(num_batches):
    original_image = X_test_single_im[batch_index]
    ground_truth_image = Y_test[batch_index]
    predicted_image_2d = test_pred_2d[batch_index]
    predicted_image_3d = test_pred_3d[batch_index]

    # Display images and calculate metrics
    display_and_calculate_metrics(original_image, predicted_image_2d, predicted_image_3d, ground_truth_image)

    # Update running totals for average metrics
    psnr_2d, ssim_2d, pcc_2d = calculate_metrics(ground_truth_image, predicted_image_2d)
    psnr_3d, ssim_3d, pcc_3d = calculate_metrics(ground_truth_image, predicted_image_3d)

    # Append metrics to lists
    psnr_2d_list.append(psnr_2d)
    ssim_2d_list.append(ssim_2d)
    pcc_2d_list.append(pcc_2d)

    psnr_3d_list.append(psnr_3d)
    ssim_3d_list.append(ssim_3d)
    pcc_3d_list.append(pcc_3d)

# Calculate average metrics across all batches
average_psnr_2d = np.mean(psnr_2d_list)
average_ssim_2d = np.mean(ssim_2d_list)
average_pcc_2d = np.mean(pcc_2d_list)

average_psnr_3d = np.mean(psnr_3d_list)
average_ssim_3d = np.mean(ssim_3d_list)
average_pcc_3d = np.mean(pcc_3d_list)

# Print average metrics
print("Average Metrics Across All Batches:")
print(f"PSNR (2D CRISPIM): {average_psnr_2d:.4f}, SSIM (2D CRISPIM): {average_ssim_2d:.4f}, PCC (2D CRISPIM): {average_pcc_2d:.4f}")
print(f"PSNR (3D CRISPIM): {average_psnr_3d:.4f}, SSIM (3D CRISPIM): {average_ssim_3d:.4f}, PCC (3D CRISPIM): {average_pcc_3d:.4f}")

# Plot graphs
image_numbers = np.arange(1, num_batches + 1)

plt.figure(figsize=(15, 5))

# PSNR
plt.subplot(1, 3, 1)
plt.plot(image_numbers, psnr_2d_list, marker='o', label='2D CRISPIM')
plt.plot(image_numbers, psnr_3d_list, marker='o', label='3D CRISPIM')
plt.xlabel('Image Number')
plt.ylabel('PSNR')
plt.legend()
plt.title('PSNR Comparison')

# SSIM
plt.subplot(1, 3, 2)
plt.plot(image_numbers, ssim_2d_list, marker='o', label='2D CRISPIM')
plt.plot(image_numbers, ssim_3d_list, marker='o', label='3D CRISPIM')
plt.xlabel('Image Number')
plt.ylabel('SSIM')
plt.legend()
plt.title('SSIM Comparison')

# PCC
plt.subplot(1, 3, 3)
plt.plot(image_numbers, pcc_2d_list, marker='o', label='2D CRISPIM')
plt.plot(image_numbers, pcc_3d_list, marker='o', label='3D CRISPIM')
plt.xlabel('Image Number')
plt.ylabel('PCC')
plt.legend()
plt.title('PCC Comparison')

plt.tight_layout()
plt.show()

